In [9]:
input_file = "mh_aggr.out"

import json
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow_probability as tfp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

import sys

# TODO: Change root path
root = "/Users/jay/Desktop/Bachelorarbeit"

sys.path.append(f"{root}/Implementation")
from src.execute_model import run_model_single_parameter_node
from src.construct_model import get_model


ndims = 7
dims = ["TT", "C0", "beta", "ETF", "FC", "FRAC", "K2"]
colors = sns.color_palette(n_colors=ndims)

testConfigPath = f"{root}/test_config.json"
with open(testConfigPath, "r") as file:
    test_config = json.load(file)

configPath = test_config["configPath"]
basis = test_config["basis"]
model = get_model(configPath, basis)

start_date: 2004-01-01 00:00:00
start_date_predictions: 2005-01-01 00:00:00
end_date: 2006-01-01 00:00:00
simulation length: 365
full_data_range is 732 hours including spin_up_length of 366 hours
simulation_range is of length 366 hours


In [2]:
# Construct params
configurationObject = model.configurationObject
param_lower = []
param_upper = []
for param in configurationObject["parameters"]:
    if param["distribution"] == "Uniform":
        param_lower.append(param["lower"])
        param_upper.append(param["upper"])
    else:
        raise NotImplementedError(
            f"Sorry, the distribution {param['distribution']} is not supported yet"
        )
param_lower = np.array(param_lower)
param_upper = np.array(param_upper)

In [3]:
samples = pd.read_csv(f"{input_file}")
samples

,TT,C0,beta,ETF,FC,FRAC,K2
0,1.653004,0.000000,2.274710,0.890849,236.803024,0.180250,0.050000
1,0.425389,0.000000,1.991245,0.657032,283.316922,0.111752,0.050000
2,-0.516615,0.000000,1.750573,0.660525,384.211939,0.100000,0.041089
3,-0.158393,0.947425,2.026865,0.504073,492.332343,0.209639,0.041510
4,-1.696926,0.000000,2.054925,0.678411,500.000000,0.191594,0.042201
...,...,...,...,...,...,...,...
7996,3.149245,7.801644,1.000000,0.525734,54.593581,0.249602,0.024575
7997,4.000000,5.535728,1.000000,0.628623,115.097656,0.293997,0.026412
7998,3.912024,5.547146,1.000000,0.501862,131.344250,0.185083,0.022135
7999,4.000000,5.978241,1.000000,0.331692,165.017088,0.315647,0.021610


# Plotting

In [11]:

fig = make_subplots(rows=2, cols=4)

# Histogram and KDE
for i, col in enumerate(samples.columns):
    row = (i // 4) + 1 
    col_idx = (i % 4) + 1 

    # Histogram
    fig.add_trace(
        go.Histogram(x=samples[col], name=col, histnorm="probability density"),
        row=row,
        col=col_idx,
    )

    # KDE calculation
    kde = gaussian_kde(samples[col])
    x_values = np.linspace(samples[col].min(), samples[col].max(), 300)
    kde_values = kde(x_values)

    fig.add_trace(
        go.Scatter(
            x=x_values, y=kde_values, mode="lines", name=f"KDE {col}", showlegend=False
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=500, 
    width=1200, 
    title_text="Metropolis Hastings with Aggregation: Parameters Overview",
)
fig.show()


In [12]:
fig = make_subplots(rows=2, cols=4)

for i, col in enumerate(samples.columns):
    row = (i // 4) + 1 
    col_idx = (i % 4) + 1  

    fig.add_trace(
        go.Box(
            y=samples[col],
            name=col,
            boxpoints="all",
            jitter=0.5,
            whiskerwidth=0.2,
            marker_size=2,
            line_width=1,
        ),
        row=row,
        col=col_idx,
    )

# Update layout 
fig.update_layout(
    height=500, 
    width=1200, 
    title_text="Metropolis Hastings with Aggregation: Boxplots of Parameters",
)
fig.show()


In [6]:
corr_matrix = samples.corr()

fig = go.Figure(
    data=go.Heatmap(
        z=corr_matrix,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale="Viridis", 
        colorbar=dict(title="Correlation Coefficient"),
    )
)

# Update layout
fig.update_layout(
    title="Metropolis Hastings with Aggregation: Heatmap of Variable Correlations",
    xaxis_title="Variables",
    yaxis_title="Variables",
)

fig.show()

# Sampling

In [7]:
# Sampling Max
param_vec = []
for i in range(len(samples.loc[0])):
    values, counts = np.unique(samples.iloc[:, i], return_counts=True)
    ind = np.argmax(counts)
    param_vec.append(values[ind])

_, posterior_max, _, _ = run_model_single_parameter_node(model, param_vec)

[HVBSASK INFO] [0] parameters: [[4.0, 10.0, 1.0, 1.0, 500.0, 0.1, 0.05]]


In [8]:
# Mean sampling from posterior
sample_param = []
for i in range(7):
    sample_param.append(np.random.choice(samples.iloc[:, i], 1000))
sample_param = np.array(sample_param).T

posterior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    posterior.append(y_model)

print(np.array(posterior).shape)
posterior_mean = np.mean(np.array(posterior), axis=0)

[HVBSASK INFO] [0] parameters: [array([2.36053803e+00, 1.00000000e+01, 2.10814740e+00, 9.11941456e-01,
       1.31756854e+02, 9.00000000e-01, 2.80693352e-02])]
[HVBSASK INFO] [0] parameters: [array([2.69997587e+00, 4.51104036e+00, 2.18981946e+00, 5.65330372e-01,
       4.53332632e+02, 1.00000000e-01, 0.00000000e+00])]
[HVBSASK INFO] [0] parameters: [array([-2.45277943e+00,  2.08797416e+00,  2.24638354e+00,  5.77876925e-01,
        4.17810623e+02,  4.02536336e-01,  1.46585860e-03])]
[HVBSASK INFO] [0] parameters: [array([-9.79235332e-01,  5.03477818e+00,  1.48075116e+00,  4.42105767e-02,
        3.39247049e+02,  1.00000000e-01,  5.00000000e-02])]
[HVBSASK INFO] [0] parameters: [array([2.97017717e+00, 1.00000000e+01, 2.24638354e+00, 2.15510810e-01,
       2.45471489e+02, 2.84362213e-01, 1.83694794e-02])]
[HVBSASK INFO] [0] parameters: [array([-2.91914444e+00,  0.00000000e+00,  1.96530018e+00,  7.27092579e-01,
        3.75122723e+02,  7.23944630e-01,  1.01797938e-02])]
[HVBSASK INFO] [0] 

In [9]:
# Mean smpling from prior
sample_param = []
for i in range(7):
    uni = tfp.distributions.Uniform(low=param_lower[i], high=param_upper[i]).sample(1)
    sample_param.append(uni)
sample_param = np.array(sample_param).T

prior = []
for _, vec in enumerate(sample_param):
    _, y_model, _, _ = run_model_single_parameter_node(model, np.array(vec))
    prior.append(y_model)

_, _, measured_data, _ = run_model_single_parameter_node(model, np.array(vec))

prior_means = np.mean(np.array(prior), axis=0)

2024-06-06 00:12:02.092592: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-06 00:12:02.092625: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-06-06 00:12:02.092631: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-06-06 00:12:02.092651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-06 00:12:02.092664: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[HVBSASK INFO] [0] parameters: [array([5.49850208e-01, 4.22867028e+00, 1.02633678e+00, 1.93427935e-01,
       1.77510432e+02, 6.60917479e-01, 4.84368388e-02])]
[HVBSASK INFO] [0] parameters: [array([5.49850208e-01, 4.22867028e+00, 1.02633678e+00, 1.93427935e-01,
       1.77510432e+02, 6.60917479e-01, 4.84368388e-02])]


In [10]:
fig = go.Figure()
model.get_start_date
dates = model.get_simulation_range()
fig.add_trace(
    go.Scatter(
        x=dates,
        y=prior_means,
        mode="lines",
        name="Prior Mean",
        line=dict(color="lightgrey"),
    )
)
fig.add_trace(
    go.Scatter(x=dates, y=posterior_mean, mode="lines", name="Posterior Mean")
)
fig.add_trace(go.Scatter(x=dates, y=posterior_max, mode="lines", name="Posterior Max"))
fig.add_trace(go.Scatter(x=dates, y=measured_data, mode="lines", name="Measured Data"))

# Update layout
fig.update_layout(
    title="Metropolis Hastings with Aggregation: Bayesian Inference Result Comparison",
    xaxis_title="Date",
    yaxis_title="Value",
    legend_title="Time Series",
    hovermode="x unified",
    template="plotly_white",
)

fig.show()

In [11]:
def rmse(result, target):
    diff = result - target
    aggr = 0
    for i in range(len(diff)):
        aggr += diff[i] ** 2
    rmse = (aggr / (len(diff))) ** 0.5
    return rmse


def mae(result, target):
    return np.absolute(result - target).mean()


print(f"RMSE of Posterior Mean: {rmse(posterior_mean, measured_data)}")
print(f"RMSE of Posterior Max: {rmse(posterior_max, measured_data)}")
print(f"MAE of Posterior Mean: {mae(posterior_mean, measured_data)}")
print(f"MAE of Posterior Max: {mae(posterior_max, measured_data)}")

RMSE of Posterior Mean: 21.61327291916241
RMSE of Posterior Max: 26.45066897160825
MAE of Posterior Mean: 11.31416388124353
MAE of Posterior Max: 10.51853137416778
